# Revised case normalization for Hirslanden Aarau 2017

- No matches were found in this hospital



In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')

import os

from py.global_configs import *
from py.normalize import normalize

from src.service import bfs_cases_db_service as bfs_db

from src.utils.dataframe_utils import remove_duplicated_chops

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:

file_info = FileInfo(
        os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
         'Hirslanden Aarau',
        '2017',
        ['Aarau_2017'])

print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI_Aarau_Birshof_ST. Anna.xlsx', hospital_name_db='Hirslanden Aarau', year='2017', sheets=['Aarau_2017'])


In [3]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

df_revised_case_d2d = normalize(file_info, 0)

2022-10-18 10:21:40.225 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 0: discarded ICDs after validation {''}
2022-10-18 10:21:40.225 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 1: discarded ICDs after validation {''}
2022-10-18 10:21:40.226 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 2: discarded ICDs after validation {''}
2022-10-18 10:21:40.226 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 3: discarded ICDs after validation {''}
2022-10-18 10:21:40.227 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 5: discarded ICDs after validation {''}
2022-10-18 10:21:40.228 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 8: discarded ICDs after validation {''}
2022-10-18 10:21:40.228 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 9: discarded ICDs after validation {''}
2022-10-18 10:21:40.229 | DEBUG    | src.utils.dataframe_utils:_validate_icd

Read 14 cases for Hirslanden Aarau 2017
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
old_pd              string
new_pd              string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [4]:
df_revised_case_d2d

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,41206174,1176423,W,72,5,3,I21C,T840,T840,M200,[E1140],[],"[80051D, 815255]",[]
1,41230776,1659346,W,67,19,3,B72C,G001,G001,M100,[B370],[],[],[]
2,41281042,1823347,M,67,6,3,F14B,E1150,E1150,M100,[F101],[],[],[]
3,41386032,1051316,M,74,10,3,I08C,S7211,S7211,M200,[L8914],[],[],[]
4,41418378,22136564,M,86,8,3,G18B,C182,C182,M200,[E1191],[E1190],[],[]
5,41247420,1251372,W,63,8,3,I09D,M4216,M4216,M200,[M4196],[],[],[]
6,41477047,1536048,W,74,6,3,I66B,M315,M315,M100,[K3182],[K5581],[],[]
7,0041280551,0022114393,W,84,7,3,F62C,I5012,I5012,M100,"[F023, F338, F051]","[F03, F328]",[],[]
8,0041457913,0001193700,W,92,5,3,E71B,C341,C341,M100,[R15],[],[],[]
9,0041523346,0002096726,W,77,9,3,L60D,N1783,N1783,M100,[A099],[],[],[]


# Match revised DtoD data with the bfs data from the database


In [5]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['41206174', '41230776', '41281042', '41386032', '41418378',
       '41247420', '41477047', '0041280551', '0041457913', '0041523346',
       '0041282182', '0041360177', '0041588330', '0041440505'],
      dtype=object)

In [6]:
revised_case_id_stripped = [case_id.lstrip('0') for case_id in revised_case_id]

### Check if there are duplicated case_ids in the revised cases after stripping the 0 in front, if yes, delete duplicated cases

In [7]:

revised_case_id_stripped_set = set(revised_case_id_stripped)
if len(revised_case_id_stripped) > len(revised_case_id_stripped_set):
    print('There are duplicated case_ids in the revised cases.' )
else:
    print('There are no duplicated case_ids in the revised cases.' )

There are no duplicated case_ids in the revised cases.


In [8]:
# match to the database and retrieve data from database using revised
revised_case_db = bfs_db.get_bfs_cases_by_ids_no_pad0(list(revised_case_id_stripped_set))
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl


In [9]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,...,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl,case_id_stripped


In [10]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()
print(revised_case_db.shape)

(0, 21)


In [11]:
# create a new column for case_id_stripped with '0' for joining to cases from database

df_revised_case_d2d['case_id_stripped']=  df_revised_case_d2d['case_id'].apply(lambda x: x.lstrip('0')) 
df_revised_case_d2d.head()
print(df_revised_case_d2d.shape)

(14, 15)


In [12]:
# delete duplicated cases in the revised cases

df_revised_case_d2d['case_id_stripped'].apply(tuple) # make sure the drop_duplicates works

df_revised_case_d2d.drop_duplicates(subset=['case_id_stripped'], keep='last', inplace=True)
assert len(revised_case_id_stripped) == df_revised_case_d2d.shape[0] # make sure all cases are deleted


# Forming the normalized dataframe: Merge the selected revised case data with the matched from bfs data
  If there are multiple cases were found based on case_id, patient_id will be used to match once more  

In [13]:
# Check if there are multiple cases were found based on case_id, patient_id will be used to match once more
if revised_case_db.shape[0] <= len(revised_case_id):
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_db.merge(revised_case_d2d_subset, how ='left', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_id = revised_case_normalized.shape[0]
else:
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_db.merge(revised_case_d2d_subset, how ='left', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_patient_id = revised_case_normalized.shape[0]
print(revised_case_normalized.shape)
revised_case_normalized.head()

(0, 28)


,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_days,clinic_id,ventilation_hours,admission_weight,gestation_age,...,added_icds,removed_icds,added_chops,removed_chops,case_id_stripped,gender,age_years,duration_of_stay,pccl,drg


In [14]:
# Choose column to export to a csv file
df_revised_case_normalized = revised_case_normalized[COLS_TO_OUTPUT]
assert len(df_revised_case_normalized.columns) == len(COLS_TO_OUTPUT)
df_revised_case_normalized.head()

,aimedic_id,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops


# Export the normalized revised cases with the neccessary columns to a csv file


In [15]:

hospital = file_info.hospital_name_db.replace(' ', '_') # replace spaces with '_' for the file name
hospital_year = f'{hospital}_{file_info.year}'

output_folder = os.path.join(ROOT_DIR, 'normalized_revision_data/')

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

output_path = os.path.join(ROOT_DIR, 'normalized_revision_data/') + hospital_year + '.csv'

df_revised_case_normalized.to_csv(output_path)


# Report

A small summary of:
- how many cases could be matched in the DB, and based on which columns
- how many cases were “correctly” revised, in terms of upcoding (still need to do after the `grouper` is available for python)

In [16]:
if revised_case_db.shape[0] <= len(revised_case_id):
    print(f'Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )
else:
    print(f'Based on the columns "case_id" and "Patient_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_patient_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )

Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are 0 out of 14 revised cases from DtoD that can be matched with the bfs database data for Hirslanden Aarau 2017


In [17]:
print('All the normalized revised cases are exported to here: {}'.format(output_path))

All the normalized revised cases are exported to here: /home/jovyan/work/src/revised_case_normalization/normalized_revision_data/Hirslanden_Aarau_2017.csv
